In [ ]:
import duckdb
import sqlite3
import sys

def sqlite_to_duckdb(sqlite_path, duckdb_path):
    # Connect to SQLite database
    sqlite_conn = sqlite3.connect(sqlite_path)
    sqlite_cursor = sqlite_conn.cursor()

    # Connect to DuckDB database
    duckdb_conn = duckdb.connect(duckdb_path)

    # Get list of tables from SQLite
    sqlite_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = sqlite_cursor.fetchall()

    # Install and load SQLite extension in DuckDB
    duckdb_conn.execute("INSTALL sqlite;")
    duckdb_conn.execute("LOAD sqlite;")

    # Attach SQLite database to DuckDB
    duckdb_conn.execute(f"ATTACH '{sqlite_path}' AS sqlite_db (TYPE SQLITE);")

    # Copy each table from SQLite to DuckDB
    for table in tables:
        table_name = table[0]
        print(f"Copying table: {table_name}")
        duckdb_conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM sqlite_db.{table_name};")

    # Close connections
    sqlite_conn.close()
    duckdb_conn.close()

    print(f"Conversion complete. DuckDB database saved to {duckdb_path}")

# if __name__ == "__main__":
#     if len(sys.argv) != 3:
#         print("Usage: python script.py <sqlite_path> <duckdb_path>")
#         sys.exit(1)
#     
#     sqlite_path = sys.argv[1]
#     duckdb_path = sys.argv[2]
sqlite_to_duckdb(r'../../data/maine_legislation_and_testimony.db', r'../../data/maine_legislation_and_testimony.duckdb')